# Automating GIS with Python
## Mapping commute to work mode choice 
---
What county or counties would you like to analyze? 
Enter the name or names of the counties in the following cell, maintaining the proper format described below.
Notice that each county listed should be enclosed in single quotes, separated by commas,
written in title case, and include the word 'County'. Enter the counties between the double brackets.
The commas used to separate the county names must fall **outside** of the quotations. Example:
```study_counties = ['Sacramento County', 'Yolo County', 'Sutter County', 'Yuba County']```
---


In [ ]:
study_counties = ['Marin County', 'San Francisco County', 'Alameda County', 'San Mateo County']

### Once you have entered the counties you would like to analyze, press Shift + Enter to execute the cell 

When you hit Shift + Enter, the selected cell will run. If it runs properly the 'run order' number will appear in the brackets to the left of the cell. If there is an error, the code will not execute, and an error message will appear below the cell. 

Now that you have chosen the counties you would like to analyze, it is time to run the code! The code is broken down into appropriate sections for review and discussion.

## Step I: Import the required libraries

In [ ]:
#import required libraries

import requests
import pandas as pd
import geopandas as gpd
import numpy as np
import geojson
import folium
import os


## Step II: Download the data from the Census API

In [ ]:

#Set up the variables used to build a call url to the the census api website

HOST = "https://api.census.gov/data"
year = "2018"
#dataset = "acs/acs5/subject"
dataset = "acs/acs5"
api_key = "f9e79198302081250c07d556f35d8a81cdae528a"
base_url = "/".join([HOST, year, dataset,])

#These 'predicates' help build the more complex query to the query. 
#Notice that the variables (columns) are specified here, the column names for our new dataframe
#are also created here. The predicate dictionary keys are then assigned values, which will be passed
#into the request.get function. 

#Setting up request for Table B08006: Sex of Workers by Means of Transportation to Work

predicates_transpo_mode = {}
get_vars_transpo_mode = ["NAME","B08006_001E","B08006_002E","B08006_003E", "B08006_004E",
                         "B08006_008E","B08006_014E","B08006_015E","B08006_016E", 
                         "B08006_017E", "GEO_ID"
                        ]
col_names_transpo_mode = ["place_name", "total","total_car_truck_van",
                          "car_truck_van_drove_alone","car_truck_van_carpooled",
                          "public_transportation", "bike", "walk", "taxi_moto_other",
                          "work_from_home","geoid","state_code", "county", "tract"
                         ]
predicates_transpo_mode["key"] = api_key
predicates_transpo_mode["get"] = ",".join(get_vars_transpo_mode)
predicates_transpo_mode["for"] = "tract:*"
predicates_transpo_mode["in"] = "state:06"
transpo_mode = requests.get(base_url, params=predicates_transpo_mode)



## Step III: Clean the data, and then join it with Census Tigerline Census Tract spatial data. 

In [5]:
#Setting up Means of Transportation to Workdata frame, getting rid of first header row
df_transpo_mode = pd.DataFrame(columns=col_names_transpo_mode, data=transpo_mode.json()[1:])

# Now we join the census data to Tigerline census tract geometries. 

#A shapefile containing all census tracts in California 
#has been manually pulled from the census ftp site, unzipped, and saved to a folder
#Census Tract Tigerline California Census Tracts location (ftp url included in notes below)

#for personal_laptop
#census_tracts_shp = "../../Users/calvindechicago/PycharmProjects/AltaWork/automated_census_mapping/tl_2018_06_tract/simplified_5ft/tl_2018_06_simplified_wgs.shp"

#for jupyter_binder : See: https://discourse.jupyter.org/t/what-is-with-the-weird-jovyan-user/1673
census_tracts_shp = "/home/jovyan/data/tl_2018_06_simplified_wgs.shp"

#I had trouble finding the file path from the Binder environment, so these lines are for troubleshooting:

#print current working directory
print(os.getcwd() + "\n")

#print whether or not a given directory exists
os.path.exists(census_tracts_shp)

#This reads the census tracts shapefile into a geodataframe
gdf = gpd.read_file(census_tracts_shp)

#the column names are all caps. We want to make them lowercase. This maps the lower function to the column names
gdf.columns = map(str.lower, gdf.columns)

#The geoid field in the df_transpo_mode table does not match the Tigerlines geoid field. 
#This slices the the right 11 most digits, which match the geoid codes in the TigerLine file. 
#(... these are state ('06') for California, followed by county, followed by census tract)
df_transpo_mode.insert(14, "geoid_join",df_transpo_mode['geoid'].str.slice(-11), True) 

#split the place_name to get human known county names
#str.split splits on comma  (',') delimiter. .str[1] selects the second element in the list (the county name) 
df_transpo_mode.insert(1, "county_name",df_transpo_mode['place_name'].str.split(',').str[1].str.strip(), True)

#this uses study_counties variable created by user in beginning of notebook!
tracts_select_counties = df_transpo_mode.loc[df_transpo_mode['county_name'].isin(study_counties)]

#JOIN the transpo_mode table with the tigerline table 
#Note:gdf must be left table, the table that merge method is run on, so that a geodataframe (not a dataframe)
#is returned. 
df_transpo_mode_with_geom = gdf.merge(tracts_select_counties,left_on='geoid',right_on='geoid_join')


#Change data types of columns that will be used for analysis.
df_transpo_mode_with_geom['bike'] = df_transpo_mode_with_geom['bike'].astype(int)
df_transpo_mode_with_geom['total'] = df_transpo_mode_with_geom['total'].astype(int)

# Create a Geo-id which is needed by the Folium (it needs to have a unique identifier for each row)
# We do not want the GeoJson object created earlier. Use original df_transpo_mode_with_geom data.
#census_tracts_gjson = folium.features.GeoJson(df_transpo_mode_with_geom, name="census tracts")
df_transpo_mode_with_geom['geoid'] = df_transpo_mode_with_geom.index.astype(str)

#calculate percentage of people that bike to work in each tract
a = ((df_transpo_mode_with_geom['bike'] / df_transpo_mode_with_geom['total'])*100).round(1)

#insert new column with calculated percentage of bike riders in each census tract
df_transpo_mode_with_geom.insert(2,'pct_bike',a, True)

# Select only needed columns
choropleth_data = df_transpo_mode_with_geom[['geoid', 'bike', 'pct_bike', 'geometry']]

# Convert to geojson (not needed for the simple coropleth map!)
#pop_json = data.to_json()

#check data
choropleth_data.head()

#This gives a warning and I'm not sure if indexing the geoid column affects functionality, 
#so I am commenting this line out for now. 
#choropleth_data['geoid'] = choropleth_data.index.astype(str)


#This finds the total bounds of the selected counties, and creates a variable with the centroid,
#which will be used to locate and center the selection within the map frame. 
bounds = df_transpo_mode_with_geom.total_bounds
a = np.mean(bounds[0:3:2]).round(3)
b = np.mean(bounds[1:4:2]).round(3)
data_centroid = [b,a]
print(data_centroid)


# STEP IV: Create the map using Folium

In [ ]:
# Create a Map instance
m = folium.Map(location=data_centroid, tiles = 'cartodbpositron', zoom_start=9, control_scale=True)

#Plot a choropleth map
#Notice: 'geoid' column that we created earlier needs to be assigned always as the first column
folium.Choropleth(
    geo_data=choropleth_data,
    name='Percentage of Cyclists',
    data=choropleth_data,
    columns=['geoid', 'pct_bike'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white',
    line_weight=0,
    highlight=False,
    smooth_factor=1.0,
    #threshold_scale=[1, 2, 3, 4, 5],
    legend_name= 'Percentage of workers that bike to work').add_to(m)

# Convert points to GeoJson
# This creates interactive labels
folium.features.GeoJson(choropleth_data,
                        name='Labels',
                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=['pct_bike'],
                                                               # aliases = ['Population'],
                                                                labels=True,
                                                                sticky=False
                                                                            )
                       ).add_to(m)


## Step V: Render the map!

In [ ]:
#Show map
print('Please wait for the map to load ...')
m

### Once you run the #Show map cell above, please wait for the map to load. 
Below are a list of sources and other notes that I kept as I wrote this script. 
Not shown are hundreds of lines of troubleshooting and test code. 

In [31]:
#SOURCES
#https://www.w3schools.com/tags/ref_urlencode.ASP
#https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf
#https://api.census.gov/data/2018/acs/acs5/variables.html
#https://api.census.gov/data/2018/acs/acs5/subject/variables.html
#https://www.youtube.com/watch?v=Wi0_Mb0e4JM
#https://atcoordinates.info/2019/09/24/examples-of-using-the-census-bureaus-api-with-python/
#--downloading tigerline from zip -- 
#http://andrewgaidus.com/Dot_Density_County_Maps/
#--Python for reading zip tigerline shpfile --
#http://andrewgaidus.com/Reading_Zipped_Shapefiles/
#--Aaron's ATP Data Mining Project would also be useful--
#https://github.com/AltaPlanning/GIS-notebooks/tree/master/2020-000%20ATP%20Data%20Mining
#https://automating-gis-processes.github.io/site/notebooks/L5/interactive-map-folium.html

#-- geographies and summary levels --
#https://censusreporter.org/topics/geography/
#geo_ids=140|04000US06  --> this should be a all tracts in California

#There is no great way to use the api to return census tract geometries: the geography api functions 
#only seem to allow calling a specific geoid. One option would be to loop through geoids and call census reporter
#to request geography for each geoid, but that would involve a lot of calls. 
#https://api.censusreporter.org/1.0/data/show/latest?table_ids=B01001&geo_ids=140|04000US06
#error"You requested 8057 geoids. The maximum is 3500. Please contact us for bulk data."



# MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS) BY MEANS OF TRANSPORTATION TO WORK
# Survey/Program: American Community Survey
# Universe: Workers 16 years and over with earnings
# Year: 2018
# Estimates: 1-Year
# Table ID: B08121



In [32]:
# ### OTHER NOTES
#I was a bit confused about obtaining the Census Tiger boundaries. The Tigerweb REST service seemed geared towards
#delivering Web Map Service (WMS) map images. We want the spatial data! Hopefully the stack exchange post linked below
#clears some of the confusion up.
#https://gis.stackexchange.com/questions/269650/how-to-bring-the-tiger-census-reporter-api-to-geopandas
#--->I'm not sure if these geometries still exist on the census api. 
#zipfiles can be downloaded at the ftp site below. I am using blog and aaron's atp data mining python as examples. 
#For now I am just going to manually unzip census geometry, but a link to a tutorial is included below to automate
#download, unzipping, and processing the geometry. 

#ftp://ftp2.census.gov/geo/tiger/TIGER2018/TRACT/tl_2018_06_tract.zip